# Demonstration of Pure RL for Computing Eigenvectors

This notebook demonstrates the feasibility of using pure Reinforcement Learning (RL) for computing eigenvectors. 

However, the resulting cosine distance ended up being 90 degrees, indicating a completely random result.

**Random pair of vectors in higher dimensional euclidian spaces are likely to be perpendicular** 
## Proof


Take $e=[1 \hspace{0.2cm} 0 \hspace{0.2cm} 0 \hspace{0.2cm} .\hspace{0.2cm}.\hspace{0.2cm}. \hspace{0.2cm} 0]^\intercal \in \mathbb{R}^n$.
If $ S $ denotes the surface measure on the unit sphere corresponding to (normalized) area, then the Expected inner product of $e$ and random variable $Y$ with support set being a unit sphere $S$ given by 
$$
\mathbb{E}[e^\intercal Y]
= \int |\mathbf{e} \cdot \mathbf{y}|^2 \, dS(\mathbf{y}) 
= \int |y_1|^2 \, dS(\mathbf{y}) 
= \frac{1}{n} \int \sum_{j=1}^n |y_j|^2 \, dS(\mathbf{y}) 
= \frac{1}{n}.
$$


So as the dimensionality $n$ increases, the expected inner product goes to $0$ ie. any random pair of vectors are likely to be orthogonal

In [2]:
!pip install 'shimmy>=2.0'
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [3]:
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from scipy.sparse import csr_matrix, issparse
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import cosine
from stable_baselines3 import SAC


In [4]:
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from scipy.sparse import csr_matrix, issparse
from scipy.sparse.linalg import eigsh
from scipy.sparse import random as sparse_random

# Power method for warm-start initialization
def power_method(A, num_iterations=5):
    x = np.random.randn(A.shape[0])
    x /= np.linalg.norm(x)
    for _ in range(num_iterations):
        x = A.dot(x) if issparse(A) else A @ x
        x /= np.linalg.norm(x) + 1e-8
    return x

# Define RL Environment for Eigenvector Search
class EigenvectorEnv(gym.Env):
    def __init__(self, A, target_eigenvalue):
        super(EigenvectorEnv, self).__init__()
        self.A = A
        self.target_eigenvalue = target_eigenvalue
        self.dim = A.shape[0]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.dim,), dtype=np.float32)
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.dim,), dtype=np.float32)
        self.state = None
        # Compute Frobenius norm for normalization
        self.A_norm = np.sqrt(np.sum(A.data**2)) if issparse(A) else np.linalg.norm(A, 'fro')

    def reset(self):
        # Warm-start with power method
        self.state = power_method(self.A)
        return self.state

    def step(self, action):
        action = np.array(action)
        action /= np.linalg.norm(action) + 1e-8
        self.state = action
        # Compute residual norm
        if issparse(self.A):
            residual = self.A.dot(action) - self.target_eigenvalue * action
        else:
            residual = self.A @ action - self.target_eigenvalue * action
        residual_norm = np.linalg.norm(residual)
        # Normalize residual norm
        normalized_residual = residual_norm / (self.A_norm + 1e-8)
        # Compute Rayleigh quotient deviation
        rayleigh = (action @ (self.A.dot(action) if issparse(self.A) else self.A @ action)) / (np.dot(action, action) + 1e-8)
        rayleigh_deviation = np.abs(rayleigh - self.target_eigenvalue) / (np.abs(self.target_eigenvalue) + 1e-8)
        # Combined reward with Rayleigh weight
        reward = -normalized_residual - 0.75 * rayleigh_deviation
        done = False
        return self.state, reward, done, {}

# Function to compute cosine distance
def cosine_distance(v1, v2):
    v1 = v1 / (np.linalg.norm(v1) + 1e-8)
    v2 = v2 / (np.linalg.norm(v2) + 1e-8)
    cos_sim = np.abs(np.dot(v1, v2))
    cos_sim = min(cos_sim, 1.0)  # Handle numerical errors
    return np.arccos(cos_sim) / np.pi  # Normalized to [0, 1/2]

# Function to find dominant eigenvalue and compute eigenvector using RL
def compute_dominant_eigenvector(A, timesteps=100000, eval_steps=750):
    # Ensure matrix is symmetric
    if issparse(A):
        A = (A + A.T) / 2
    else:
        A = (A + A.T) / 2

    # Compute dominant eigenvalue and eigenvector using scipy
    eigvals, eigvecs = eigsh(A, k=1, which='LA') if issparse(A) else np.linalg.eigh(A)
    dominant_eigenvalue = eigvals[-1] if not issparse(A) else eigvals[0]
    true_eigenvector = eigvecs[:, -1] if not issparse(A) else eigvecs[:, 0]
    print("Dominant eigenvalue (scipy):", dominant_eigenvalue)

    # Initialize environment and PPO model with deeper MLP
    env = EigenvectorEnv(A, dominant_eigenvalue)
    policy_kwargs = dict(net_arch=[512, 512, 256, 256, 128])
    model = PPO(
        'MlpPolicy',
        env,
        verbose=0,  # Reduce logging for multiple runs
        learning_rate=0.00005,
        batch_size=256,
        clip_range=0.1,
        policy_kwargs=policy_kwargs
    )
    model.learn(total_timesteps=timesteps)

    # Evaluate to find best eigenvector
    obs = env.reset()
    best_reward = -np.inf
    best_vec = None

    for _ in range(eval_steps):
        action, _ = model.predict(obs)
        obs, reward, _, _ = env.step(action)
        if reward > best_reward:
            best_reward = reward
            best_vec = obs

    #print("Approximated eigenvector (RL):", best_vec)
    # Compute residual norm for verification
    residual_norm = np.linalg.norm(A @ best_vec - dominant_eigenvalue * best_vec) if not issparse(A) else np.linalg.norm(A.dot(best_vec) - dominant_eigenvalue * best_vec)
    print("Residual norm ||Ax - λx||:", residual_norm)
    # Compute cosine distance for accuracy
    cos_dist = cosine_distance(best_vec, true_eigenvector)
    print("Cosine distance to true eigenvector:", cos_dist)

    return dominant_eigenvalue, best_vec, cos_dist


# Test on Large sparse matrix

In [ ]:

# Test loop: Run 10 times and store results
A_sparse_list = []
eigenvalue_list = []
eigenvector_list = []
cos_dist_list = []

for i in range(10):
    size = 1000 + i*100
    print(f"\nRun {i+1}/10 (size={size})")
    # Reset seed for reproducibility
    np.random.seed(42)

    A_sparse = sparse_random(size, size, density=0.01, format='csr')

    # Store input matrix
    A_sparse_list.append(A_sparse)

    # Run test
    eigenvalue, eigenvector, cos_dist = compute_dominant_eigenvector(A_sparse)

    # Store results
    eigenvalue_list.append(eigenvalue)
    eigenvector_list.append(eigenvector)
    cos_dist_list.append(cos_dist)

# Print cosine distances and compute mean
print("\nCosine Distances for 10 Runs:")
for i, cos_dist in enumerate(cos_dist_list):
    print(f"Run {i+1}: {cos_dist:.6f}")

mean_cos_dist = np.mean(cos_dist_list)
print(f"Mean Cosine Distance: {mean_cos_dist:.6f}")


Run 1/10 (size=1000)
Dominant eigenvalue (scipy): 5.3497582686143055


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Residual norm ||Ax - λx||: 5.299645422499358
Cosine distance to true eigenvector: 0.490538798289205

Run 2/10 (size=1100)
Dominant eigenvalue (scipy): 5.8644707923857196


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Residual norm ||Ax - λx||: 5.855986294364944
Cosine distance to true eigenvector: 0.49484522769028366

Run 3/10 (size=1200)
Dominant eigenvalue (scipy): 6.362035776208715
Residual norm ||Ax - λx||: 6.3432100397954025
Cosine distance to true eigenvector: 0.48580628423920563

Run 4/10 (size=1300)
Dominant eigenvalue (scipy): 6.8439218149729975
Residual norm ||Ax - λx||: 6.83578156325586
Cosine distance to true eigenvector: 0.46894021054551466

Run 5/10 (size=1400)
Dominant eigenvalue (scipy): 7.308640773589529
Residual norm ||Ax - λx||: 7.291900778167563
Cosine distance to true eigenvector: 0.4940616616233107

Run 6/10 (size=1500)
Dominant eigenvalue (scipy): 7.799504478874313
Residual norm ||Ax - λx||: 7.801881269191708
Cosine distance to true eigenvector: 0.4883043474066261

Run 7/10 (size=1600)
Dominant eigenvalue (scipy): 8.305845707464508
Residual norm ||Ax - λx||: 8.279084187672415
Cosine distance to true eigenvector: 0.4986708729753089

Run 8/10 (size=1700)
Dominant eigenvalue (sc

# Test on dense matrix

In [ ]:
# Test with dense symmetric matrix
np.random.seed(42)
size = 5
A_dense = np.random.randn(size, size)
A_dense = (A_dense + A_dense.T) / 2
print("\nTesting with dense matrix:")
eigenvalue, eigenvector, cos_dist_dense = compute_dominant_eigenvector(A_dense)




Testing with dense matrix:
Dominant eigenvalue (scipy): 1.991113406258497
Residual norm ||Ax - λx||: 0.5061796599950376
Cosine distance to true eigenvector: 0.09034343783723195


In [7]:

# Test loop: Run 10 times and store results
A_sparse_list1 = []
eigenvalue_list1 = []
eigenvector_list1 = []
cos_dist_list1 = []

for i in range(1,5):
    size = 10**i
    print(f"\nRun {i+1}/10 (size={size})")
    # Reset seed for reproducibility
    np.random.seed(42)

    A_sparse = sparse_random(size, size, density=0.01, format='csr')

    # Store input matrix
    A_sparse_list1.append(A_sparse)

    # Run test
    eigenvalue, eigenvector, cos_dist = compute_dominant_eigenvector(A_sparse)

    # Store results
    eigenvalue_list1.append(eigenvalue)
    eigenvector_list1.append(eigenvector)
    cos_dist_list1.append(cos_dist)

# Print cosine distances and compute mean
print("\nCosine Distances for 10 Runs:")
for i, cos_dist in enumerate(cos_dist_list1):
    print(f"Run {i}: {cos_dist:.6f}")

mean_cos_dist1 = np.mean(cos_dist_list1)
print(f"Mean Cosine Distance: {mean_cos_dist1:.6f}")


Run 2/10 (size=10)
Dominant eigenvalue (scipy): 0.0994212020444026


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Residual norm ||Ax - λx||: 0.06626127946896695
Cosine distance to true eigenvector: 0.22589007282572202

Run 3/10 (size=100)
Dominant eigenvalue (scipy): 1.045437354313167
Residual norm ||Ax - λx||: 0.9402528564586157
Cosine distance to true eigenvector: 0.4394658374983439

Run 4/10 (size=1000)
Dominant eigenvalue (scipy): 5.349758268614306
Residual norm ||Ax - λx||: 5.321606511467171
Cosine distance to true eigenvector: 0.48834816175342505

Run 5/10 (size=10000)
Dominant eigenvalue (scipy): 50.34612208476445
Residual norm ||Ax - λx||: 50.325597232190255
Cosine distance to true eigenvector: 0.49840532269518223

Cosine Distances for 10 Runs:


NameError: name 'cos_dist_list' is not defined

In [11]:
print(2*np.array(cos_dist_list1))

[0.45178015 0.87893167 0.97669632 0.99681065]



metric used cosine_dist = $arccos(|cos(\theta)|)/\pi$   
ranges from $0$ (colinear) to $\frac{1}{2}$ (orthogonal)  
  
Run 2 (size=10)  
Dominant eigenvalue (scipy): 0.0994212020444026  
Residual norm ||Ax - λx||: 0.06626127946896695  
Cosine distance to true eigenvector: 0.45178015  
  
Run 3 (size=100)  
Dominant eigenvalue (scipy): 1.045437354313167  
Residual norm ||Ax - λx||: 0.9402528564586157  
Cosine distance to true eigenvector: 0.87893167  
  
Run 4 (size=1000)  
Dominant eigenvalue (scipy): 5.349758268614306  
Residual norm ||Ax - λx||: 5.321606511467171  
Cosine distance to true eigenvector: 0.97669632   
  
Run 5 (size=10000)  
Dominant eigenvalue (scipy): 50.34612208476445  
Residual norm ||Ax - λx||: 50.325597232190255  
Cosine distance to true eigenvector: 0.99681065  
  
